# Finding Lane Lines on the Road 

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

**Some OpenCV functions that might be useful:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

In [2]:
slope_left_prior = -0.5
slope_right_prior = 0.5
x_left_prior = 0
y_left_prior = 0
x_right_prior = 0
y_right_prior = 0

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    global slope_left_prior, slope_right_prior, x_left_prior, y_left_prior, x_right_prior, y_right_prior
    
    #set initial parameters for the line
    slope_left = []
    slope_right = []
    x_left_int = []
    y_left_int = []
    x_right_int = []
    y_right_int = []
   
    for line in lines:
        for cord in line:
            if cord[2]-cord[0] != 0:
                slope = (float(cord[3]-cord[1]))/(float(cord[2]-cord[0]))
                if -0.8 < slope < -0.3:
                    slope_left.append(slope) 
                    y_left_int.append((cord[1]+cord[3])/2)
                    x_left_int.append((cord[0]+cord[2])/2)
                if 0.3 < slope < 0.8:
                    slope_right.append(slope) 
                    y_right_int.append((cord[1]+cord[3])/2)
                    x_right_int.append((cord[0]+cord[2])/2)
    
    #use the mean of the slopes to extrapolate the line 
    if len(slope_left) == 0:
        slope_left_mean = slope_left_prior
        x_left_mean = x_left_prior
        y_left_mean = y_left_prior
    else:
        slope_left_mean = sum(slope_left)/float(len(slope_left))
        if slope_left_prior * 1.25 < slope_left_mean < slope_left_prior * 0.75:
            slope_left_mean = 0.9*slope_left_mean + 0.1*slope_left_prior
        else:
            slope_left_mean = 0.1*slope_left_mean + 0.9*slope_left_prior
        slope_left_prior = slope_left_mean
        x_left_mean = sum(x_left_int)/float(len(x_left_int))
        y_left_mean = sum(y_left_int)/float(len(y_left_int))
        x_left_prior = x_left_mean
        y_left_prior = y_left_mean
    
    if len(slope_right) == 0:
        slope_right_mean = slope_right_prior
        x_right_mean = x_right_prior
        y_right_mean = y_right_prior
    else:
        slope_right_mean = sum(slope_right)/float(len(slope_right))
        if slope_right_prior * 0.75 < slope_right_mean < slope_right_prior * 1.25:
            slope_right_mean = 0.9*slope_right_mean + 0.1*slope_right_prior
        else:
            slope_right_mean = 0.1*slope_right_mean + 0.9*slope_right_prior
        slope_right_prior = slope_right_mean
        x_right_mean = sum(x_right_int)/float(len(x_right_int))
        y_right_mean = sum(y_right_int)/float(len(y_right_int))
        x_right_prior = x_right_mean
        y_right_prior = y_right_mean
    
    x1_left = int(x_left_mean - (y_left_mean - img.shape[0])/slope_left_mean)
    x2_left = int((region_of_interest_y - img.shape[0])/slope_left_mean + x1_left)
    x1_right = int(x_right_mean - (y_right_mean - img.shape[0])/slope_right_mean)
    x2_right = int((region_of_interest_y - img.shape[0])/slope_right_mean + x1_right)
    
    cv2.line(img, (x1_left, img.shape[0]), (x2_left, region_of_interest_y), color, thickness)
    cv2.line(img, (x1_right, img.shape[0]), (x2_right, region_of_interest_y), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α, β, λ):
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [3]:
# Test parameters
gaussian_blur_kernel_size = 5
canny_low_threshold = 50
canny_high_threshold = 150
region_of_interest_y = 330
region_of_interest_x_left = 450
region_of_interest_x_right = 490
hough_lines_rho = 2
hough_lines_theta = np.pi/180
hough_lines_threshold = 20
hough_lines_min_line_length = 40
hough_lines_max_line_gap = 5
weighted_img_α = 1.
weighted_img_β = 1.
weighted_img_λ = 0.

# Test function
def process_image(image):
    gray = grayscale(image)
    blur_gray = gaussian_blur(gray, gaussian_blur_kernel_size)
    edges = canny(blur_gray, canny_low_threshold, canny_high_threshold)
    region_of_interest_vertices = np.array([[(0,image.shape[0]),(region_of_interest_x_left, region_of_interest_y), 
                                             (region_of_interest_x_right, region_of_interest_y), 
                                             (image.shape[1],image.shape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, region_of_interest_vertices)
    line = hough_lines(masked_edges, hough_lines_rho, hough_lines_theta, hough_lines_threshold, 
                       hough_lines_min_line_length, hough_lines_max_line_gap)
    result = weighted_img(line, image, weighted_img_α, weighted_img_β, weighted_img_λ)
    return result

## Test on Images

In [4]:
import os
os.listdir("test_images/")

['p_solidWhiteCurve.jpg',
 'p_solidWhiteRight.jpg',
 'p_solidYellowCurve.jpg',
 'p_solidYellowCurve2.jpg',
 'p_solidYellowLeft.jpg',
 'p_whiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg',
 'solidWhiteRight.jpg',
 'solidYellowCurve.jpg',
 'solidYellowCurve2.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg']

In [5]:
p_solidWhiteCurve = process_image(mpimg.imread('test_images/solidWhiteCurve.jpg'))
mpimg.imsave('test_images/p_solidWhiteCurve.jpg', p_solidWhiteCurve)
p_solidWhiteRight = process_image(mpimg.imread('test_images/solidWhiteRight.jpg'))
mpimg.imsave('test_images/p_solidWhiteRight.jpg', p_solidWhiteRight)
p_solidYellowCurve = process_image(mpimg.imread('test_images/solidYellowCurve.jpg'))
mpimg.imsave('test_images/p_solidYellowCurve.jpg', p_solidYellowCurve)
p_solidYellowCurve2 = process_image(mpimg.imread('test_images/solidYellowCurve2.jpg'))
mpimg.imsave('test_images/p_solidYellowCurve2.jpg', p_solidYellowCurve2)
p_solidYellowLeft = process_image(mpimg.imread('test_images/solidYellowLeft.jpg'))
mpimg.imsave('test_images/p_solidYellowLeft.jpg', p_solidYellowLeft)
p_whiteCarLaneSwitch = process_image(mpimg.imread('test_images/whiteCarLaneSwitch.jpg'))
mpimg.imsave('test_images/p_whiteCarLaneSwitch.jpg', p_whiteCarLaneSwitch)

## Test on Videos

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
white_output = 'white.mp4'
clip1 = VideoFileClip('solidWhiteRight.mp4')
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 97.41it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.36 s, sys: 491 ms, total: 2.85 s
Wall time: 2.47 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [9]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:07<00:00, 93.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 7.75 s, sys: 1.52 s, total: 9.26 s
Wall time: 7.64 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

###### Where will your current algorithm be likely to fail?
1. My current algorithm has a set region of interest. If the camera angle changes, the region has to be updated accordingly. That also means this will not work when there are hills or sharp turns. 
2. The slopes and coordinates setting were trial and error. They work for the two videos and for most of the challenge below. But it is not a robust solution. 
3. Actually, all test parameters in the third block are not robust enough for a work-for-all solution.
4. No robust distortion correction or color thresholding.

###### How could you imagine making your algorithm better / more robust?
Having a more robust definition for region of interest, range for test parameters, slope and coordinate and better distortion correction and color threshold will improve the current algorithm. There are a couple of frames where the current code doesn't work well for the optional challenge, like when the car drove onto the patch of road with lighter colors. I believe this is covered in the later modules. 

## Optional Challenge

In [11]:
region_of_interest_y = 450
region_of_interest_x_left = 575
region_of_interest_x_right = 750
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)
#challenge_clip = clip2.fl_image(plt.imshow)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:05<00:00, 45.17it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 4.97 s, sys: 1.25 s, total: 6.22 s
Wall time: 6.28 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))